# Kaggle Amazon

В этом туториале показана основная функциональность библиотеки CatBoost с использованием датасета Amazon.

Полную верcию этого ноутбука можно найти в официальном [репозитории](https://github.com/catboost/tutorials) с туториалами CatBoost.


Когда сотрудник любой компании приступает к работе, ему сначала нужно получить доступ к компьютеру, необходимый для выполнения своей роли. Этот доступ может позволить сотруднику читать/манипулировать ресурсами через различные приложения или веб-порталы. Предполагается, что сотрудники, выполняющие функции определенной роли, будут иметь доступ к одним и тем же или аналогичным ресурсам. Часто бывает так, что сотрудники выясняют, какой доступ им нужен, когда сталкиваются с препятствиями в своей повседневной работе (например, не могут войти на портал отчетности). Тогда опытный руководитель тратит время на то, чтобы вручную предоставить необходимый доступ, чтобы преодолеть препятствия. Поскольку сотрудники перемещаются по компании, этот цикл поиска/восстановления доступа отнимает нетривиальное количество времени и денег.

Существует значительный объем данных о роли сотрудника в организации и ресурсах, к которым он имеет доступ. Учитывая данные о текущих сотрудниках и предоставленном им доступе, можно построить модели, которые будут автоматически определять привилегии доступа по мере того, как сотрудники будут входить и выходить из ролей в компании. Эти модели автоматического доступа призваны свести к минимуму участие человека, необходимое для предоставления или отзыва доступа сотрудников.

Задача данного исследования - построить модель, обучаемую на основе исторических данных, которая будет определять потребности сотрудника в доступе. 
Модель будет принимать информацию о роли сотрудника и код ресурса и выдавать ответ, следует ли предоставлять доступ к этому ресурсу.


# Чтение данных

In [1]:
import pandas as pd
train_df = pd.read_csv('amazon/train.csv')
test_df = pd.read_csv('amazon/test.csv')

In [2]:
train_df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


# Подготовка датасета

Выделение целевой переменной

In [3]:
y = train_df.ACTION
X = train_df.drop('ACTION', axis=1)

Объявление категориальных факторов

In [4]:
cat_features = range(0, X.shape[1])
print (cat_features)

range(0, 9)


# Обучение модели

Разделение данных на train и validation

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

Обучение модели

In [6]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    thread_count=2,
    iterations=5,
    random_seed=1136926949945377,
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent'
)

In [7]:
print (model.random_seed_)

1136926949945377


In [8]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    thread_count=2,
    iterations=300,
    learning_rate=0.1,
    random_seed=63,
    custom_loss=['AUC', 'Accuracy'],
    use_best_model=True
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [9]:
print ('Model is fitted:', model.is_fitted())

Model is fitted: True


In [10]:
print ('Model params:', model.get_params())

Model params: {'iterations': 300, 'learning_rate': 0.1, 'thread_count': 2, 'random_seed': 63, 'use_best_model': True, 'custom_loss': ['AUC', 'Accuracy']}


In [11]:
print ('Resulting tree count:', model.tree_count_)

Resulting tree count: 235


# Способы задания датасета

In [12]:
from catboost.utils import create_cd
import os

feature_names = dict()
for column, name in enumerate(train_df):
    if column == 0:
        continue
    feature_names[column] = name
    
create_cd(
    label=0, 
    cat_features=list(range(1, train_df.columns.shape[0])),
    feature_names=feature_names,
    output_path='amazon/test.cd'
)

In [13]:
!cat amazon/test.cd

"cat" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [15]:
import numpy as np
from catboost import Pool
pool1 = Pool(data=X, label=y, cat_features=cat_features)
pool2 = Pool(data='amazon/train.csv', delimiter=',', has_header=True, column_description='amazon/test.cd', thread_count=2)

print ('Dataset shape')

print ('dataset 1:', pool1.shape, '\ndataset 2:', pool2.shape)


print ('Column names')
print ('dataset 1:', pool1.get_feature_names(), '\ndataset 2:',  pool1.get_feature_names())

Dataset shape
dataset 1: (32769, 9) 
dataset 2: (32769, 9)
Column names
dataset 1: ['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE'] 
dataset 2: ['RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']


# Кросс-валидация

In [17]:
from catboost import cv

params = {}
params['loss_function'] = 'Logloss'
params['iterations'] = 80
params['custom_loss'] = 'AUC'
params['random_seed'] = 63
params['learning_rate'] = 0.5

cv_data = cv(
    params = params,
    pool = Pool(X, label=y, cat_features=cat_features),
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=False,
    verbose=False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.1674879003
bestIteration = 36

Training on fold [1/5]

bestTest = 0.164632916
bestIteration = 48

Training on fold [2/5]

bestTest = 0.1533395872
bestIteration = 79

Training on fold [3/5]

bestTest = 0.1426916182
bestIteration = 78

Training on fold [4/5]

bestTest = 0.1563234371
bestIteration = 37



In [20]:
import numpy as np
np.set_printoptions(precision=3)

for name, values in cv_data.items():
    print (name + ':')
    print (np.array(values))
    print ('\n')

iterations:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79]


test-Logloss-mean:
[0.302 0.227 0.19  0.178 0.172 0.168 0.165 0.164 0.163 0.163 0.162 0.161
 0.161 0.161 0.16  0.16  0.16  0.16  0.16  0.16  0.16  0.159 0.159 0.159
 0.159 0.159 0.158 0.159 0.158 0.158 0.158 0.159 0.158 0.158 0.158 0.158
 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.158
 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.159 0.159 0.159 0.159
 0.158 0.158 0.158 0.158 0.158 0.158 0.158 0.159 0.159 0.159 0.159 0.159
 0.159 0.159 0.159 0.158 0.158 0.158 0.158 0.158]


test-Logloss-std:
[0.004 0.008 0.006 0.007 0.007 0.007 0.008 0.008 0.008 0.008 0.008 0.008
 0.008 0.007 0.007 0.007 0.007 0.007 0.007 0.008 0.008 0.007 0.008 0.007
 0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.008 0

In [21]:
best_value = np.max(cv_data['test-AUC-mean'])
best_iter = np.argmax(cv_data['test-AUC-mean'])
print ('Best validation AUC score: {:.2f}±{:.2f} on step {}'.format(
    best_value,
    cv_data['test-AUC-std'][best_iter],
    best_iter
))

Best validation AUC score: 0.86±0.01 on step 39


# Подбор параметров

In [22]:
best_model = CatBoostClassifier(
    iterations=1500,
    learning_rate=0.01,
    l2_leaf_reg=3,
    bagging_temperature=1,
    random_strength=1,
    one_hot_max_size=0,
    random_seed=63,
    use_best_model=True
)
best_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

## Детектор переобучения

In [23]:
model_full = CatBoostClassifier(
    eval_metric='AUC',
    learning_rate=0.8,
    iterations=500,
    random_seed=42
)
model_full.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [24]:
model_with_earlystop = CatBoostClassifier(
    eval_metric='AUC',
    learning_rate=0.8,
    iterations=500,
    random_seed=42,
    od_type='Iter',
    od_wait=20
)

model_with_earlystop.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [25]:
model_with_trunk = CatBoostClassifier(
    eval_metric='AUC',
    learning_rate=0.8,
    iterations=500,
    random_seed=42,
    od_type='Iter',
    od_wait=20,
    use_best_model=True
)

model_with_trunk.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [26]:
print ('Original model tree count:', model.tree_count_)
print ('Early-stopped model tree count:', model_with_earlystop.tree_count_)
print ('Trunkated model tree count:', model_with_trunk.tree_count_)

Original model tree count: 235
Early-stopped model tree count: 17
Trunkated model tree count: 17


## Сравнение нескольких моделей

In [27]:
model1 = CatBoostClassifier(
    learning_rate=0.9,
    iterations=100,
    train_dir='learing_rate_0.9',
    name='learing_rate_0.9'
)

model2 = CatBoostClassifier(
    learning_rate=0.1,
    iterations=100,
    train_dir='learing_rate_0.1',
    name='learning_rate_0.1'
)

In [28]:
model1.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Verbose'
)
model2.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Verbose'
)

0:	learn: 0.2766015	test: 0.2773871	best: 0.2773871 (0)	total: 20.6ms	remaining: 2.04s
1:	learn: 0.2211736	test: 0.2219929	best: 0.2219929 (1)	total: 47.9ms	remaining: 2.35s
2:	learn: 0.1928008	test: 0.1836306	best: 0.1836306 (2)	total: 75.1ms	remaining: 2.43s
3:	learn: 0.1871643	test: 0.1775831	best: 0.1775831 (3)	total: 100ms	remaining: 2.4s
4:	learn: 0.1822635	test: 0.1728024	best: 0.1728024 (4)	total: 126ms	remaining: 2.39s
5:	learn: 0.1792784	test: 0.1686609	best: 0.1686609 (5)	total: 151ms	remaining: 2.36s
6:	learn: 0.1785669	test: 0.1685862	best: 0.1685862 (6)	total: 177ms	remaining: 2.35s
7:	learn: 0.1766385	test: 0.1654709	best: 0.1654709 (7)	total: 201ms	remaining: 2.31s
8:	learn: 0.1754529	test: 0.1654948	best: 0.1654709 (7)	total: 226ms	remaining: 2.29s
9:	learn: 0.1745383	test: 0.1652455	best: 0.1652455 (9)	total: 263ms	remaining: 2.37s
10:	learn: 0.1732515	test: 0.1644654	best: 0.1644654 (10)	total: 300ms	remaining: 2.43s
11:	learn: 0.1726837	test: 0.1642513	best: 0.16425

In [29]:
from catboost import MetricVisualizer
widget = MetricVisualizer(['learing_rate_0.9', 'learing_rate_0.1'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Вычисление метрик на новом датасете

In [30]:
tree_count = model.tree_count_
metrics = model.eval_metrics(pool1, metrics=['Logloss','AUC','Accuracy'], ntree_start=0, ntree_end=0,
                             eval_period=tree_count, thread_count=2)
auc = metrics['AUC']
print (auc)

[0.4998336013697382, 0.9834073159800195]


# Важность факторов

In [31]:
# Найдем самые важные факторы
importances = best_model.feature_importances_
print ('Feature importances:', np.array(importances))
print ('Feature names:', np.array(pool1.get_feature_names()))

Feature importances: [22.689 16.494  4.728  8.927 17.648  8.115 10.893  4.563  5.942]
Feature names: ['RESOURCE' 'MGR_ID' 'ROLE_ROLLUP_1' 'ROLE_ROLLUP_2' 'ROLE_DEPTNAME'
 'ROLE_TITLE' 'ROLE_FAMILY_DESC' 'ROLE_FAMILY' 'ROLE_CODE']


# Сохранение модели

In [32]:
my_best_model = CatBoostClassifier(iterations=10)
my_best_model.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Verbose'
)

Learning rate set to 0.5
0:	learn: 0.3965403	test: 0.3968906	best: 0.3968906 (0)	total: 21ms	remaining: 189ms
1:	learn: 0.2940955	test: 0.2943532	best: 0.2943532 (1)	total: 49.1ms	remaining: 197ms
2:	learn: 0.2433303	test: 0.2443831	best: 0.2443831 (2)	total: 76.4ms	remaining: 178ms
3:	learn: 0.2193310	test: 0.2210140	best: 0.2210140 (3)	total: 106ms	remaining: 160ms
4:	learn: 0.1981216	test: 0.1951440	best: 0.1951440 (4)	total: 137ms	remaining: 137ms
5:	learn: 0.1894261	test: 0.1841792	best: 0.1841792 (5)	total: 164ms	remaining: 109ms
6:	learn: 0.1830562	test: 0.1763128	best: 0.1763128 (6)	total: 189ms	remaining: 81.2ms
7:	learn: 0.1799460	test: 0.1725612	best: 0.1725612 (7)	total: 220ms	remaining: 54.9ms
8:	learn: 0.1779373	test: 0.1702787	best: 0.1702787 (8)	total: 247ms	remaining: 27.4ms
9:	learn: 0.1766187	test: 0.1682470	best: 0.1682470 (9)	total: 273ms	remaining: 0us

bestTest = 0.1682470454
bestIteration = 9



In [33]:
my_best_model.save_model('catboost_model.bin')

In [34]:
my_best_model.load_model('catboost_model.bin')
print (my_best_model.get_params())

{'iterations': 10, 'loss_function': 'Logloss', 'logging_level': 'Verbose'}
